In [1]:

import numpy as np
import scipy as sp
import h5py

import os

import SimData as sim_data
import Simulator as sim_system
import Optimization as opt

In [2]:


###* Input files and function

reactions_file = "reactionsSimpleV1.json"

const_dict = {
        "F0": 1.5e15,           # cm^-2
        "S0": 3e13,             # cm^-2
        
        "R": 0.00831442,        # kJ/mol*K
        "kBoltz": 1.380649e-23, # J/K
}

initial_state_dict = {'O_F': 0.1, 'O2_F':0.1 ,'O_S': 0.1, 'Vdb_S':0.1, 
                      'Odb_S': 0.1, 'CO_F': 0.1, 'CO2_F':0.1, 'CO_S': 0.1, 
                      'COdb_S': 0.0}

###* Functions for the data transformation
def compute_flux(const_dict, exp_dict, specie, molar_mass):
    den = exp_dict.get(specie, 0.0)
    v_th = np.sqrt((8.0 * const_dict['R'] * 1000 * exp_dict['Tnw'])/(molar_mass * np.pi))
    flux = 0.25 * v_th * den * 100
    return flux


def compute_remaining_flux(const_dict, exp_dict, molar_mass): 
    den = exp_dict['N'] - exp_dict['O'] - exp_dict['CO']
    v_th = np.sqrt((8.0 * const_dict['R'] * 1000 * exp_dict['Tnw'])/(molar_mass * np.pi))
    flux = 0.25 * v_th * den * 100
    return flux

####? EavgMB data extracted from the Booth et al. 2019 paper
p_data_exp = [0.2, 0.3, 0.4, 0.5, 0.6, 0.75, 1.5]
EavgMB_data = [1.04, 0.91, 0.87, 0.83, 0.77, 0.5, 0.001]
interpolator = sp.interpolate.interp1d(p_data_exp, EavgMB_data, kind='linear', fill_value=0.001, bounds_error=False)


transformations_exp = {
    'Tw':       lambda const_dict, exp_dict: exp_dict['Tw'] + 273.15,
    'fluxO' :   lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict,'O', 0.016),
    'fluxO2' :  lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict,'O2', 0.032),
    'fluxO3' :  lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict,'O3', 0.048),
    'fluxC':    lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'C', 0.012),
    'fluxCO':   lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'CO', 0.028),
    'fluxCO2':  lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'CO2', 0.048),
    'EavgMB':   lambda const_dict, exp_dict: interpolator(exp_dict['pressure']).item(),
    'Ion':      lambda const_dict, exp_dict: 1e14 * exp_dict["current"]
}

output_folder_path = "Buffer_Data"
exp_data_file = "Experimental_data_CO_Jorge.hdf5"

In [3]:


def func_optimization(params):
    
    SF_1, SF_2, SF_3, SF_4, SF_5, SF_6, SF_7, SF_8 = params
    
    A = 2.777e-02
    B = 7.792e-04
    E = 1.627e+01
    
    nu_d_mod = lambda T: 1e15 * (A + B * np.exp(E/(const_dict['R'] * T)))
    
    dict_mod_vec = [
    {"id": 2, "rate": None, "model_dict": {"nu_d": nu_d_mod}},
    {"id": 10, "rate": None, "model_dict": {"nu_d": nu_d_mod}},
    {"id": 31, "rate": None, "model_dict": {"nu_d": nu_d_mod}},
    
    {"id": 32, "rate": None, "model_dict": {"SF": SF_1}},
    {"id": 35, "rate": None, "model_dict": {"SF": SF_2}},
    
    {"id": 33, "rate": None, "model_dict": {"SF": SF_3}},
    {"id": 34, "rate": None, "model_dict": {"SF": SF_4}},
    
    {"id": 36, "rate": None, "model_dict": {"SF": SF_5}},
    {"id": 37, "rate": None, "model_dict": {"SF": SF_6}},
    
    {"id": 38, "rate": None, "model_dict": {"SF": SF_7}},
    {"id": 39, "rate": None, "model_dict": {"SF": SF_8}},
    ]
    
    return dict_mod_vec

loss_function = lambda exp, teo: np.mean((np.reciprocal(exp)*(exp-teo))**2)

exp_file = os.path.join(output_folder_path, exp_data_file)
sim = sim_system.Simulator(reactions_file, const_dict, exp_file, initial_state_dict, transformations_exp=transformations_exp)


  d[CO2_F]/dt = -CO2_F*r_29 + r_28*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[CO_F]/dt = -CO_F*O_F*r_34 - 0.02*CO_F*O_S*r_39 - CO_F*r_31 - CO_F*r_33 - 0.02*CO_F*r_35*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) + r_30*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[CO_S]/dt = CO_F*r_35*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) - CO_S*O_F*r_38 - CO_S*r_36 + r_32*(-CO_S - O_S - Odb_S - Vdb_S + 1.0)
  d[O2_F]/dt = -O2_F*O_F*r_15 - O2_F*r_10 - O2_F*r_12 - O2_F*r_14 + r_9*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[O_F]/dt = -CO_F*O_F*r_34 - 0.02*CO_S*O_F*r_38 - O2_F*O_F*r_15 - 2*O_F**2*r_8 - 0.02*O_F*O_S*r_7 - 0.02*O_F*Odb_S*r_27 - 0.02*O_F*Vdb_S*r_26 - O_F*r_11 - O_F*r_2 - O_F*r_4 - 0.02*O_F*r_5*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) + r_1*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[O_S]/dt = -CO_F*O_S*r_39 - O_F*O_S*r_7 + O_F*r_5*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) - O_S*r_16 - O_S*r_17 - O_S*r_37 - O_S*r_6 + r_3*(-CO_S - O_S - Odb_S - Vdb_S + 1.0)
  d[Odb_S]/dt = -O_F*Odb_S*r_27 + O_F*Vdb_S*r_26 - Odb_S*r_23 - Odb_S*r_24 - Odb

In [4]:

if __name__ == '__main__':
    optimizer = opt.Optimizer(sim, func_optimization, loss_function)

    global_bounds = [
                    (1e-3, 5e-2), (5e-2, 0.5), (1e-3, 9e-2), 
                    (1e-3, 9e-2), (6e-2, 6e-1), (9e-3, 9e-2),
                    (5e-3, 9e-2), (5e-2, 0.3)
                    ]

    config = {
        "bounds": global_bounds,
        "de_maxiter": 3,
        "de_num_iterations": 1,
    }
    
    best_local, best_local_loss = optimizer.hybrid_optimization_searchV2(config)

# differential_evolution step 50: f(x)= 0.041662857183863554


differential_evolution step 1: f(x)= 0.2713657004235628
differential_evolution step 2: f(x)= 0.18866985102513506
differential_evolution step 3: f(x)= 0.18866985102513506
differential_evolution step 4: f(x)= 0.06597974647731862
differential_evolution step 5: f(x)= 0.06597974647731862
differential_evolution step 6: f(x)= 0.06597974647731862
differential_evolution step 7: f(x)= 0.06597974647731862
differential_evolution step 8: f(x)= 0.06597974647731862
differential_evolution step 9: f(x)= 0.06597974647731862
differential_evolution step 10: f(x)= 0.06597974647731862
differential_evolution step 11: f(x)= 0.06597974647731862
differential_evolution step 12: f(x)= 0.06597974647731862
differential_evolution step 13: f(x)= 0.06597974647731862
differential_evolution step 14: f(x)= 0.06597974647731862
differential_evolution step 15: f(x)= 0.06597974647731862
differential_evolution step 16: f(x)= 0.06597974647731862
differential_evolution step 17: f(x)= 0.05345341143925609


differential_evolution step 18: f(x)= 0.05345341143925609
differential_evolution step 19: f(x)= 0.05345341143925609
differential_evolution step 20: f(x)= 0.05345341143925609
differential_evolution step 21: f(x)= 0.05345341143925609


differential_evolution step 22: f(x)= 0.05345341143925609
differential_evolution step 23: f(x)= 0.05345341143925609


differential_evolution step 24: f(x)= 0.05345341143925609


differential_evolution step 25: f(x)= 0.05345341143925609
differential_evolution step 26: f(x)= 0.05345341143925609
differential_evolution step 27: f(x)= 0.05345341143925609


...

differential_evolution step 70: f(x)= 0.03990645523548077
differential_evolution step 71: f(x)= 0.03990645523548077


differential_evolution step 72: f(x)= 0.03990645523548077
differential_evolution step 73: f(x)= 0.03990645523548077
differential_evolution step 74: f(x)= 0.03990645523548077


differential_evolution step 75: f(x)= 0.039837296180146566
differential_evolution step 76: f(x)= 0.039837296180146566


differential_evolution step 77: f(x)= 0.03976537157685732
differential_evolution step 78: f(x)= 0.03976537157685732
differential_evolution step 79: f(x)= 0.03976537157685732


differential_evolution step 80: f(x)= 0.03949713152277596
top_k_mod:  16
res:  0.03934810310642824 [4.79404257e-03 1.22857832e-03 1.52683612e+01 1.69054395e-02
 1.59783230e-01 2.22468373e-01 3.18892097e-01 4.83961008e-01
 3.56806504e-01 1.37776713e-02 1.44218322e-01] [4.79404257e-03 1.22857832e-03 1.52683612e+01 1.69054395e-02
 1.59783230e-01 2.22468373e-01 3.18892097e-01 4.60915246e-01
 3.56806504e-01 1.37776713e-02 1.44218322e-01]
res:  0.0397104058760094 [1.51581371e-03 1.04834687e-03 1.56189473e+01 1.73375664e-02
 1.65980379e-01 1.38912816e-01 1.18681599e-01 3.73353505e-01
 9.56618143e-02 6.26590598e-02 2.43657408e-01] [1.51581371e-03 1.04834687e-03 1.56189473e+01 1.73375664e-02
 1.65980379e-01 1.38912816e-01 1.18681599e-01 3.55574767e-01
 9.56618143e-02 6.26590598e-02 2.43657408e-01]
res:  0.039725751015714225 [3.41395362e-02 9.67368508e-04 1.57633938e+01 1.13604468e-02
 2.54867806e-01 1.55183564e-01 2.56272521e-01 3.85166372e-01
 2.02016718e-01 7.25498626e-02 1.45467317e-01] [3.4

In [5]:
print(best_local)
print(best_local_loss)

             message: Maximum number of iterations has been exceeded.
             success: False
                 fun: 0.03949713152277596
                   x: [ 4.794e-03  1.229e-03  1.527e+01  1.691e-02
                        1.598e-01  2.225e-01  3.189e-01  4.609e-01
                        3.568e-01  1.378e-02  1.442e-01]
                 nit: 80
                nfev: 13365
          population: [[ 4.794e-03  1.229e-03 ...  1.378e-02  1.442e-01]
                       [ 1.401e-02  1.217e-03 ...  2.308e-01  1.197e-01]
                       ...
                       [ 1.442e-02  1.334e-03 ...  4.212e-01  4.467e-01]
                       [ 3.534e-03  1.582e-03 ...  2.885e-03  3.463e-01]]
 population_energies: [ 3.950e-02  4.124e-02 ...  4.297e-02  4.052e-02]
             message: Optimization terminated successfully.
             success: True
                 fun: 0.038806833423079785
                   x: [ 2.777e-02  7.792e-04  1.627e+01  1.165e-02
                        1.4